# СБОР ДАННЫХ

В данном ноутбуке:

- Сбор общей информации о квартирах с каждой страницы сайта
- Сбор подробной информации со страниц описания квартир с помощью id квартиры
- Оформления всех данных в таблицу
- Сохранение исходных данных
- Описание исходных данных

## Оглавление

1. [Сбор общей информации о квартирах с каждой страницы сайта](#1)<br>
2. [Сбор подробной информации со страниц описания квартир с помощью id квартиры](#2)<br>
3. [Оформления всех данных в таблицу](#3)<br>
4. [Сохранение исходных данных](#4)<br>
5. [Описание исходных данных](#5)<br>

In [1]:
# импорт необходимых библиотек
import json
from bs4 import BeautifulSoup 
import requests
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import time
from tqdm import tqdm
import headers
import url

## Сбор общей информации о квартирах с каждой страницы сайта <a class="anchor" id="1"></a>

In [2]:
headers = headers.headers
url = url.url

In [3]:
# сбор данных с каждой старницы сайта
price = []
rooms = []
area = []
floor = []
districttr = []
lon = []
lat = []
flatId = []
floors_in_house = []
keep = []
balcon = []
building_year = []
wall_id = []
euro = []
house_address_number = []
mortgage = []
young_family = []
transport = []
street = []
for offset in tqdm(range(0,1900,30)):
    data = requests.get(url,headers=headers)
    for flat in range(0,len(data.json()['data'])):
        price.append(data.json()['data'][flat]['price'])
        rooms.append(data.json()['data'][flat]['rooms'])
        area.append(data.json()['data'][flat]['square'])
        floor.append(data.json()['data'][flat]['floor'])
        districttr.append(data.json()['data'][flat]['districttr'])
        lon.append(data.json()['data'][flat]['lo'])
        lat.append(data.json()['data'][flat]['la'])
        flatId.append(data.json()['data'][flat]['object_id'])
        floors_in_house.append(data.json()['data'][flat]['floors'])
        keep.append(data.json()['data'][flat]['keep'])
        balcon.append(data.json()['data'][flat]['balcon'])
        building_year.append(data.json()['data'][flat]['building_year'])
        wall_id.append(data.json()['data'][flat]['wall_id'])
        euro.append(data.json()['data'][flat]['euro'])
        house_address_number.append(data.json()['data'][flat]['house_address_number'])
        mortgage.append(data.json()['data'][flat]['mortgage'])
        young_family.append(data.json()['data'][flat]['young_family'])
        transport.append(data.json()['data'][flat]['transport'])
        street.append(data.json()['data'][flat]['meta']['street'])
    time.sleep(1)

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [01:58<00:00,  1.85s/it]


In [4]:
#Записываем, полученные данные в таблицу
info = {'price':price,
    'rooms':rooms,
    'area':area,
    'floor':floor,
    'districttr':districttr,
    'lon':lon,
    'lat':lat,
    'flatId':flatId,
    'floors_in_house':floors_in_house,
    'keep':keep,
    'balcon':balcon,
    'building_year':building_year,
    'wall_id':wall_id,
    'euro':euro,
    'street':street,
    'house_address_number':house_address_number,
    'mortgage':mortgage,
    'young_family':young_family,
    'transport':transport}
flats = pd.DataFrame(info)

In [5]:
flats.head()

,price,rooms,area,floor,districttr,lon,lat,flatId,floors_in_house,keep,balcon,building_year,wall_id,euro,street,house_address_number,mortgage,young_family,transport
0,7800000,3,59.4,2,zmeinka,131.895842,43.086803,8056913,5,cosmetic,yes_balcon,1971,11,True,Зои Космодемьянской,38,True,False,True
1,6500000,3,57.5,5,balyaeva,None,None,7110701,5,half,yes_balcon,1962,11,False,Адмирала Юмашева,14,True,False,True
2,8300000,1,38.7,2,centr,131.892905,43.120511,6988438,9,cosmetic,no_balcon,1996,11,False,Прапорщика Комарова,48,False,False,False
3,13950000,3,89.6,16,nekrasovskaya,131.903083,43.122281,6656303,26,good,yes_balcon,2018,64,False,Гоголя,33,True,False,True
4,5700000,1,29.3,7,tolstogo,131.913809,43.120971,5594677,13,cosmetic,yes_balcon,2020,15,False,Толстого,28,False,False,True


## Сбор подробной информации со страниц описания квартир с помощью id квартиры <a class="anchor" id="2"></a>

In [9]:
# отправляем запрос на страницу описания каждой квартиры по ее id и собираем
# данные с этой страницы
extra_info = []
browser = webdriver.Chrome(executable_path="D:/chromedriver.exe")
for flatId in tqdm(flats['flatId']):
    browser.get(f"https://vl.etagi.com/realty/{flatId}/")
    button_xpath = '//*[@id="characteristics"]/div/div/button'
    but = browser.find_elements(by=By.XPATH, value=button_xpath)
    but[0].click()
    xpath = '//*[@id="characteristics"]/div/div'
    elements = browser.find_elements(value=xpath,by=By.XPATH)
    for element in elements:
        extra_info.append(element.text)
    time.sleep(1)
browser.close()

<ipython-input-9-4e32401dc034>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="D:/chromedriver.exe")
100%|████████████████████████████████████████████████████████████████████████████| 1920/1920 [1:29:30<00:00,  2.80s/it]


## Оформления всех данных в таблицу <a class="anchor" id="3"></a>

In [10]:
def find(where_find,what_find):
    where_find = where_find.split('\n')
    try:
        return where_find[where_find.index(what_find)+1]
    except:
        return None

In [11]:
# Обработка данных, полученных со страницы описания каждой квартиры  
ceiling_height = []
renovation = []
bath = []
wall_type = []
balcon2 = []
balcon_glass = []
mater_window = []
sunny = []
view = []
lift = []
yard = []
parking = []
wate_heating_system = []
domofon = []
iron_door = []
Water_meter = [] #счетчик воды
furniture = []
technic = []
services = []
for flat in tqdm(range(len(extra_info))):
    ceiling_height.append(find(extra_info[flat],'Высота потолков'))
    renovation.append(find(extra_info[flat],'Ремонт'))
    bath.append(find(extra_info[flat],'Санузел'))
    wall_type.append(find(extra_info[flat],'Стены'))
    balcon2.append(find(extra_info[flat],'Балкон'))
    balcon_glass.append(find(extra_info[flat],'Остекление балкона'))
    mater_window.append(find(extra_info[flat],'Материал окон'))
    sunny.append(find(extra_info[flat],'Солнечная сторона'))
    view.append(find(extra_info[flat],'Вид из окон'))
    lift.append(find(extra_info[flat],'Лифт'))
    yard.append(find(extra_info[flat],'Двор'))
    parking.append(find(extra_info[flat],'Парковка'))
    wate_heating_system.append(find(extra_info[flat],'Система водонагрева'))
    domofon.append(find(extra_info[flat],'Домофон'))
    iron_door.append(find(extra_info[flat],'Железная дверь'))
    Water_meter.append(find(extra_info[flat],'Счетчик воды'))
    furniture.append(find(extra_info[flat],'Из мебели в квартире остается'))
    technic.append(find(extra_info[flat],'Из техники в квартире остается'))
    services.append(find(extra_info[flat],'Подключённые сервисы'))

100%|████████████████████████████████████████████████████████████████████████████| 1920/1920 [00:00<00:00, 6269.68it/s]


In [12]:
# добавляем, полученные данные в общую таблицу
flats['ceiling_height'] = ceiling_height
flats['renovation'] = renovation
flats['bath'] = bath
flats['wall_type'] = wall_type
flats['balcon2'] = balcon2
flats['balcon_glass'] = balcon_glass
flats['mater_window'] = mater_window
flats['sunny'] = sunny
flats['view'] = view
flats['lift'] = lift
flats['yard'] = yard
flats['parking'] = parking
flats['wate_heating_system'] = wate_heating_system
flats['domofon'] = domofon
flats['iron_door'] = iron_door
flats['Water_meter'] = Water_meter
flats['furniture'] = furniture
flats['technic'] = technic
flats['services'] = services

In [13]:
flats.head()

,price,rooms,area,floor,districttr,lon,lat,flatId,floors_in_house,keep,...,lift,yard,parking,wate_heating_system,domofon,iron_door,Water_meter,furniture,technic,services
0,7800000,3,59.4,2,zmeinka,131.895842,43.086803,8056913,5,cosmetic,...,None,Двор,наземный паркинг,None,да,есть,есть,None,None,"телефон, интернет, кабельное телевидение"
1,6500000,3,57.5,5,balyaeva,None,None,7110701,5,half,...,None,Двор,наземный паркинг,None,да,есть,есть,"кухонный гарнитур, шкаф-купе, стол рабочий, ст...",None,кабельное телевидение
2,8300000,1,38.7,2,centr,131.892905,43.120511,6988438,9,cosmetic,...,пассажирский,Двор,наземный паркинг,да,да,есть,есть,"кухонный гарнитур, шкаф-купе, стол рабочий, ст...",None,"интернет, кабельное телевидение"
3,13950000,3,89.6,16,nekrasovskaya,131.903083,43.122281,6656303,26,good,...,пассажирский и грузовой,Двор,наземный паркинг,None,да,есть,есть,None,None,"телефон, интернет, кабельное телевидение"
4,5700000,1,29.3,7,tolstogo,131.913809,43.120971,5594677,13,cosmetic,...,None,Двор,наземный паркинг,None,да,есть,есть,None,None,"телефон, интернет, кабельное телевидение"


## Сохранение исходных данных  <a class="anchor" id="4"></a>

In [14]:
# Сохраняем данные в excel файл
flats.to_excel('InitialData.xlsx')

## Описание исходных данных <a class="anchor" id="5"></a>

**Характеристики, которые были собраны:**
- price - стоимость квартиры;
- rooms - количество комнат;
- area - площадь квартиры;
- floor - этаж;
- districttr - район;
- lon - долгота;
- lat - широта;
- flatId - Id квартиры;
- floors_in_house - этажность дома;
- keep - ремонт;
- balcon - наличие балкона;
- building_year - год постройки дома;
- wall_id - тип стен дома;
- euro - является ли квартира евро-кваритрой;
- street - улица; 
- house_address_number - номер дома;
- mortgage - подходит ли под ипотеку;
- young_family - подходит ли молодой семье;
- transport - наличие транспорта;
- ceiling_height - высота потолков;
- renovation - ремонт;
- bath - наличие ванной;
- wall_type - тип стен;
- balcon2 - тип балкона;
- balcon_glass - застеклен ли балкон;
- mater_window - материал окон;
- sunny - солнечная ли сторона;
- view - вид из окна;
- lift - наличие лифта;
- yard - наличие спортивной площадки;
- parking - наличие парковки;
- wate_heating_system - наличие водонагревательной системы;
- domofon - наличие домофона;
- iron_door - наличие железной двери;
- Water_meter - наличие счетчиков;
- furniture - наличие мебели;
- technic - наличие техники;
- services - наличие сервисов (телефон, интернет и т.д.)